In [2]:
import pandas as pd
import numpy as np
import time
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *

In [3]:
# A method to fix the API
def fixAPI(API):
    if API[0]=='5':
        API = '0'+API
    return API

In [6]:
# Reading the data
start = time.time()
production_data_train = pd.read_csv("production_data_train.csv",dtype={'API':str})
# production_data_test = pd.read_csv("production_data_test.csv",dtype={'API':str})
end = time.time()
print('ReadData',end-start)

ReadData 0.5360305309295654


In [7]:
# Fix api
start = time.time()
production_data_train["API"] = production_data_train["API"].apply(fixAPI)
end = time.time()
print('FixApi',end-start)

FixApi 0.09400510787963867


In [8]:
# Removing duplicates
start = time.time()
production_data_train = production_data_train.sort_values(['_LastUpdate'])
production_data_train.drop_duplicates(subset=['API','Year','Month'],keep='last',inplace=True)
end = time.time()
print('RemovingDuplicates',end-start)

RemovingDuplicates 0.5250298976898193


In [9]:
# Remove pre-peek month
start = time.time()
maxLiquid = production_data_train.groupby(['API'],as_index=False).agg({'Liquid':'max'})
maxDate = pd.merge(maxLiquid[['API','Liquid']],production_data_train[['API','Liquid','Year','Month']],on=['API','Liquid'],how='left')

max1 = maxDate.groupby(['API','Liquid'],as_index=False).agg({'Year':'min'})
maxDate = pd.merge(maxDate[['API','Liquid','Year','Month']],max1[['API','Liquid','Year']],on=['API','Liquid','Year'],how='right')
maxDate = maxDate.groupby(['API','Liquid','Year'],as_index=False).agg({'Month':'min'})

maxDate = maxDate[['API','Year','Month']]
maxDate = maxDate.rename(columns={"Year":"maxYear",'Month':'maxMonth'})
production_data_train = pd.merge(production_data_train,maxDate[['API','maxYear','maxMonth']],on='API')
production_data_train = production_data_train[((production_data_train['Year']-production_data_train['maxYear'])*12+production_data_train['Month']-production_data_train['maxMonth'])>=0]

end = time.time()
print('RemovePrePeekMonth',end-start)

RemovePrePeekMonth 0.5060291290283203


In [10]:
# Adding Index
start = time.time()
production_data_train["Index"] = (production_data_train["Year"]-production_data_train['maxYear'])*12+production_data_train["Month"]-production_data_train["maxMonth"]+1
end = time.time()
print('IndexColumn',end-start)

IndexColumn 0.0740044116973877
